<a href="https://colab.research.google.com/github/Merrex/Repo_Runner-CLI/blob/master/repo_runner_colab_minimal_test00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimal Agentic Repo Runner Test
Test any codebase with a single click.

In [1]:
# 1. Install repo_runner (latest from GitHub)
# The tool handles dependency conflicts automatically
!pip install --upgrade --force-reinstall git+https://github.com/Merrex/Repo_Runner-CLI.git

# Simple verification
import subprocess
try:
    result = subprocess.run(['repo_runner', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ repo_runner installed: {result.stdout.strip()}")
    else:
        print("✅ repo_runner installed (CLI check failed, but module available)")
except:
    print("✅ repo_runner installed")

  Cloning https://github.com/Merrex/Repo_Runner-CLI.git to /tmp/pip-req-build-tu8059sc
  Running command git clone --filter=blob:none --quiet https://github.com/Merrex/Repo_Runner-CLI.git /tmp/pip-req-build-tu8059sc
  Resolved https://github.com/Merrex/Repo_Runner-CLI.git to commit cf8f634b222ff5710f8a8db665b17350da239d16
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached transformers-4.53.1-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached accelerate-1.8.1-py3-none-any.whl.metadata (19 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached psutil-7.0.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.

✅ repo_runner installed


In [1]:
# Step 1: Generate template
!repo_runner config

# Step 2: Rename template to .env (like local workflow)
!mv .env.template .env

# Step 3: Create .env with your tokens
with open('.env', 'w') as f:
    f.write('''NGROK_AUTH_TOKEN=2zETPURF77kinVYKr82n7V3hLDB_qLg6TCpFKhUqdXeHyv2R''')

# Step 4: Verify
!repo_runner debug

⚠️ python-dotenv not available - will use environment variables only
⚠️ transformers/torch not available: Could not import module 'pipeline'. Are this object's requirements defined correctly?
🔧 Configuration Management
✅ Created configuration template: .env.template
📝 Edit this file with your actual tokens and settings
✅ Created configuration template: .env.template
📝 Edit this file with your actual tokens and settings
🔧 Configuration Summary

🤖 Model Configuration:
  detection_agent: default (microsoft/DialoGPT-medium) ❌
  requirements_agent: default (microsoft/DialoGPT-medium) ❌
  setup_agent: default (microsoft/DialoGPT-medium) ❌
  fixer_agent: default (microsoft/DialoGPT-medium) ❌
  db_agent: default (microsoft/DialoGPT-medium) ❌
  health_agent: default (microsoft/DialoGPT-small) ❌
  runner_agent: default (microsoft/DialoGPT-medium) ❌

🔑 API Keys:
  openai: ❌
  anthropic: ❌
  google: ❌
  huggingface: ❌
  github: ❌
  gitlab: ✅

🔌 Integrations:
  ngrok: ✅
  cloudflare: ❌
  dockerhub:

In [2]:
# 2. Intelligent repository handling - supports both zip uploads and git URLs
import os
import subprocess
import zipfile
import shutil
from google.colab import files

# Configuration
test_repo_path = '/content/test_repo'
git_repo_url = None  # Set this to a git URL if you want to clone instead of upload

# Example git URLs (uncomment one to use):
# git_repo_url = 'https://github.com/username/repo-name.git'
# git_repo_url = 'https://github.com/facebook/react.git'
# git_repo_url = 'https://github.com/django/django.git'

def handle_git_repo(repo_url, target_path):
    """Intelligently handle git repository - clone if not exists, pull if exists"""
    if os.path.exists(target_path):
        print(f"📁 Repository exists at {target_path}")
        try:
            # Check if it's a git repository
            result = subprocess.run(['git', 'status'], cwd=target_path, capture_output=True, text=True)
            if result.returncode == 0:
                print("🔄 Pulling latest changes...")
                subprocess.run(['git', 'pull'], cwd=target_path, check=True)
                print("✅ Repository updated successfully")
            else:
                print("⚠️ Directory exists but not a git repo, removing and cloning...")
                shutil.rmtree(target_path)
                subprocess.run(['git', 'clone', repo_url, target_path], check=True)
                print("✅ Repository cloned successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Git operation failed: {e}")
            return False
    else:
        print(f"📥 Cloning repository from {repo_url}...")
        try:
            subprocess.run(['git', 'clone', repo_url, target_path], check=True)
            print("✅ Repository cloned successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Git clone failed: {e}")
            return False
    return True

def handle_zip_upload():
    """Handle zip file upload and extraction"""
    print("📤 Please upload your test repository zip file...")
    uploaded = files.upload()

    # Clean up existing directory to prevent duplicates
    if os.path.exists(test_repo_path):
        shutil.rmtree(test_repo_path)

    for filename in uploaded:
        if filename.endswith('.zip'):
            print(f"📦 Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(test_repo_path)
            print(f"✅ Extracted to {test_repo_path}")
            return True

    print("❌ No zip file found in uploads")
    return False

# Handle repository based on configuration
if git_repo_url:
    print(f"🔗 Using git repository: {git_repo_url}")
    success = handle_git_repo(git_repo_url, test_repo_path)
    if not success:
        print("❌ Failed to handle git repository, falling back to zip upload")
        success = handle_zip_upload()
else:
    print("📤 Using zip file upload")
    success = handle_zip_upload()

if not success:
    raise Exception("Failed to obtain test repository")

print(f"\n📁 Repository prepared at: {test_repo_path}")
print("🤖 Agent will automatically detect the actual repo root and key files")
print("🎯 Ready to test!")

📤 Using zip file upload
📤 Please upload your test repository zip file...


Saving realtime-chat-gemini-main.zip to realtime-chat-gemini-main (1).zip
📦 Extracting realtime-chat-gemini-main (1).zip...
✅ Extracted to /content/test_repo

📁 Repository prepared at: /content/test_repo
🤖 Agent will automatically detect the actual repo root and key files
🎯 Ready to test!


In [3]:
# 3. Run repo_runner fully agentically
import subprocess
import time

print("🚀 Starting repo_runner in fully agentic mode...")
print(f"📂 Target repository: {test_repo_path}")
print("⏱️ This may take a few minutes for complex repositories...")
print("🤖 Agent will automatically:")
print("   • Find the actual repository root")
print("   • Detect all services and dependencies")
print("   • Set up the environment")
print("   • Start and orchestrate services")

start_time = time.time()

try:
    result = subprocess.run(
        ['repo_runner', 'run', test_repo_path, '--mode', 'local'],
        capture_output=True,
        text=True,
        timeout=600  # 10 minute timeout
    )

    elapsed_time = time.time() - start_time
    print(f"\n⏱️ Execution time: {elapsed_time:.1f} seconds")
    print(f"Exit code: {result.returncode}")

    if result.returncode == 0:
        print("✅ repo_runner completed successfully!")
    else:
        print("❌ repo_runner encountered issues")

    print("\n--- STDOUT (last 3000 chars) ---")
    print(result.stdout[-3000:])

    if result.stderr:
        print("\n--- STDERR (last 2000 chars) ---")
        print(result.stderr[-2000:])

except subprocess.TimeoutExpired:
    print("⏰ repo_runner timed out after 10 minutes")
except Exception as e:
    print(f"❌ Error running repo_runner: {e}")

print("\n🎯 Test completed!")

🚀 Starting repo_runner in fully agentic mode...
📂 Target repository: /content/test_repo
⏱️ This may take a few minutes for complex repositories...
🤖 Agent will automatically:
   • Find the actual repository root
   • Detect all services and dependencies
   • Set up the environment
   • Start and orchestrate services

⏱️ Execution time: 15.7 seconds
Exit code: 1
❌ repo_runner encountered issues

--- STDOUT (last 3000 chars) ---
✅ Local access setup: http://localhost:3000
🔌 Setting up port access for port 3000 in colab environment
⚠️ No NGROK_AUTH_TOKEN found - using free tier (limited)
💡 Get free token at: https://dashboard.ngrok.com/get-started/your-authtoken
❌ Failed to setup Colab access: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
💡 To fix ngrok